# CMPE 549: Bioinformatics, Fall 2023
## Pairwise Sequence Global Alignment



In [ ]:
import random
import requests
import numpy as np
import pandas as pd

from Bio.Align import substitution_matrices
from Bio import Align

### Needleman Wunsch (Global Sequence Alignment)

In [ ]:
def needleman_wunsch_algorithm_linear_penalty(seq1, seq2, penalty_params):
    
    '''
    This function has 3 parts: initialisation of the main matrix, filling the main matrix and traceback matrix.

    '''
#PART 1: INITIALIZATION 
#Matrix is built with a row and column +1 longer than sequence lengths to start matrix with 0. 
  
    main_matrix = np.zeros((len(seq1)+1, len(seq2)+1))
    
#Then first row and column is filled with gap penalty scores.

    for i in range(1, len(seq1)+1):
        main_matrix[i][0] = main_matrix[i-1][0] + penalty_params["gap_penalty"]
    for j in range(1, len(seq2)+1):
        main_matrix[0][j] = main_matrix[0][j-1] + penalty_params["gap_penalty"]
        
#PART 2: FILLING        
#Matrix should be filled by getting the max value among diagonal,left and top approaches.
#Match and mismatch scores are provided from BLOSUM62 Scoring Matrix. Penalty parameters are reached
#from the dictionary penalty_params.
#Last value of main matrix will give the score, which is main_matrix[-1][-1].

    for i in range(1, len(seq1)+1):
        for j in range(1, len(seq2)+1):
            diagonal_match = main_matrix[i-1][j-1] + penalty_params["scoring_matrix"][seq1[i-1]][seq2[j-1]]
            top_linear_gap = main_matrix[i-1][j] + penalty_params["gap_penalty"]
            left_linear_gap = main_matrix[i][j-1] + penalty_params["gap_penalty"]
            
            main_matrix[i][j] = max(diagonal_match, top_linear_gap,left_linear_gap)
    
    
#PART 3: TRACEBACK
#To get the aligned sequences, path of the max value should be traced back.
#If the current score matches the sum of the last data and its score (or gap is proven),
#letter or '-' is appended to the strings seq1_aligned and seq2_aligned, which are 
#reversed and returned at the end of the function.
    
    seq1_aligned = ""
    seq2_aligned = ""
        
    i = len(seq1)
    j = len(seq2)

#Traceback continues until i and j reaches 0.

    while i>0 and j>0:
                                                                   
        score_diagonal = main_matrix[i-1][j-1]
        score_up = main_matrix[i-1][j]
        score_left = main_matrix[i][j-1]
                                                                    
        if main_matrix[i][j] == score_diagonal + penalty_params["scoring_matrix"][seq1[i-1]][seq2[j-1]]:
            seq1_aligned+= seq1[i-1]
            seq2_aligned+= seq2[j-1]
            i = i-1
            j = j-1
        elif main_matrix[i][j] == score_up + penalty_params["gap_penalty"]:
            seq1_aligned+= seq1[i-1]
            seq2_aligned+= "-"
            i = i-1
                                         
        elif main_matrix[i][j] == score_left + penalty_params["gap_penalty"]:
            seq1_aligned+= "-"
            seq2_aligned+= seq2[j-1]
            j =j-1

#In the loop above,if i or j is 0 alone, it can exit the loop.
#So this is an additional traceback to cover these situations.

    while i > 0:
        seq1_aligned += seq1[i-1]
        seq2_aligned += '-'
        i -= 1
    while j > 0:
        seq1_aligned += '-'
        seq2_aligned += seq2[j-1]
        j -= 1
        
#Score and aligned sequences are returned.

    return main_matrix[-1,-1],seq1_aligned[::-1]+ "\n" +seq2_aligned[::-1]



def needleman_wunsch_algorithm_affine_penalty(seq1, seq2, penalty_params):
    
    '''
    This function has 3 parts: initialisation of middle,lower and upper matrices, filling the matrices 
    and traceback matrix.
    In the affine penalty situation, we consider 3 layers for the matrix and switch between them while filling.
    Main level is for diagonal edges.
    Lower level is for horizontal edges.
    Upper level is for vertical edges. 
    It also includes gap extension penalty which is different from gap penalty.

    '''
#PART 1: INITIALIZATION

    gap_penalty = penalty_params['gap_penalty']
    gap_extension_penalty = penalty_params['gap_extension_penalty']
    scoring_matrix = penalty_params['scoring_matrix']
    
    mid_matrix = np.zeros((len(seq1) + 1, len(seq2) + 1))
    lower_matrix = np.zeros((len(seq1) + 1, len(seq2) + 1))
    upper_matrix = np.zeros((len(seq1) + 1, len(seq2) + 1))

#First Column: Middle and upper matrices are taken as infinity value for first column. Lower matrix is filled
#with gap penalty which continues with gap extension penalty throughout the column.

    for i in range(1, len(seq1) + 1):
        mid_matrix[i, 0] = -float('inf')
        lower_matrix[i, 0] = gap_penalty - (i - 1) * gap_extension_penalty
        upper_matrix[i, 0] = -float('inf')
        
#First Row: Middle and lower matrices are taken as infinity value for first row. Upper matrix is filled 
#with gap penalty which continues with gap extension penalty throughout the row. 

    for j in range(1, len(seq2) + 1):
        mid_matrix[0, j] = -float('inf')
        lower_matrix[0, j] = -float('inf')
        upper_matrix[0, j] = gap_penalty - (j - 1) * gap_extension_penalty

#PART 2: FILLING  
#Again, the max value of diagonal,left and top approaches are considered. The switches from lower to middle
#or upper to middle has the gap penalty. If the gap occurs in the same layer, it has gap extension penalty.
#Match and mismatch scores are provided from BLOSUM62 Scoring Matrix. Penalty parameters are reached
#from the dictionary penalty_params.
#Last value of middle matrix will give the score, which is mid_matrix[-1][-1].


    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match_score = mid_matrix[i - 1, j - 1] + scoring_matrix[seq1[i-1]][seq2[j-1]]
            lower_matrix[i, j] = max(lower_matrix[i - 1, j] + gap_extension_penalty, mid_matrix[i - 1, j] + gap_penalty)
            upper_matrix[i, j] = max(upper_matrix[i, j - 1] + gap_extension_penalty, mid_matrix[i, j - 1] + gap_penalty)
            mid_matrix[i, j] = max(match_score, lower_matrix[i, j], upper_matrix[i, j])


#PART 3: TRACEBACK
#To get the aligned sequences, path of the max value should be traced back.
#If the current score matches the sum of the last data and its score (or if the switch between
#layers are proven), letter or '-' is appended to the strings seq1_aligned and seq2_aligned, which are 
#reversed and returned at the end of the function.
    
    i = len(seq1)
    j = len(seq2)
    seq1_aligned = ""
    seq2_aligned = ""

    while i > 0 or j > 0:
        if i > 0 and j > 0 and mid_matrix[i, j] == mid_matrix[i - 1, j - 1] + scoring_matrix[seq1[i-1]][seq2[j-1]]:
            seq1_aligned+= seq1[i-1]
            seq2_aligned+= seq2[j-1]
            i -= 1
            j -= 1
        elif i > 0 and mid_matrix[i, j] == lower_matrix[i, j]:
            seq1_aligned+= seq1[i-1]
            seq2_aligned+= '-'
            i -= 1
        elif j > 0 and mid_matrix[i, j] == upper_matrix[i, j]:
            seq1_aligned+= '-'
            seq2_aligned+= seq2[j-1]
            j -= 1

    return mid_matrix[-1, -1], (seq1_aligned[::-1]+"\n"+seq2_aligned[::-1])

### BLOSUM62 Scoring Matrix

Run the cell below to obtain the scoring matrix.

In [ ]:
resp = requests.get(
    "https://raw.githubusercontent.com/rdpstaff/AlignmentTools/master/src/data/blosum62.txt"
)
_scoring_matrix = np.stack(
    [
        np.fromstring(line[2:].strip(), sep=" ")
        for line in resp.text.split("\n")
        if line and (line[0] not in ["#", " "])
    ]
)
_amino_acids = resp.text.split("\n")[6].strip().split("  ")


scoring_matrix = pd.DataFrame(
    _scoring_matrix, index=_amino_acids, columns=_amino_acids, dtype=int
)

### Generic Sequence Alignment Function

In [ ]:
def align_sequences(seq1, seq2, penalty, penalty_params):
    if penalty == "linear":
        return needleman_wunsch_algorithm_linear_penalty(seq1, seq2, penalty_params)
    elif penalty == "affine":
        return needleman_wunsch_algorithm_affine_penalty(seq1, seq2, penalty_params)
    else:
        raise NotImplementedError

### Inputs

You can use the functions below to generate sample sequences to test your algorithms. 

In [ ]:
aminoacids = [
    "A",
    "R",
    "N",
    "D",
    "C",
    "Q",
    "E",
    "G",
    "H",
    "I",
    "L",
    "K",
    "M",
    "F",
    "P",
    "S",
    "T",
    "W",
    "Y",
    "V",
]


def generate_sequence(length=50):
    return "".join([random.choice(aminoacids) for i in range(length)])


def mutate_sequence(seq, n_mutations=10):
    seq = list(seq)
    pos = {
        random.randint(1, len(seq)): random.choice(["substitute", "delete"])
        for i in range(n_mutations)
    }
    mutated_sequence = ""
    for ix, aminoacid in enumerate(seq):
        if ix in pos:
            if pos[ix] == "substitute":
                mutated_sequence += random.choice(aminoacids)
        else:
            mutated_sequence += aminoacid
    return mutated_sequence


sample_sequence = generate_sequence()
sequence1 = mutate_sequence(sample_sequence)
sequence2 = mutate_sequence(sample_sequence)
sequence1, sequence2

#### Sample 1

In [ ]:
sample1_sequence1 = "PLEASANTLY"
sample1_sequence2 = "MEANLY"

In [ ]:
def align_sequences_biopython(seq1, seq2, penalty_params):
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    if penalty_params["penalty_type"] == "linear":
        aligner.open_gap_score = penalty_params["gap_penalty"]  # -1
        aligner.extend_gap_score = penalty_params["gap_penalty"]  # -1
    elif penalty_params["penalty_type"] == "affine":
        aligner.open_gap_score = penalty_params["gap_penalty"]  # -10
        aligner.extend_gap_score = penalty_params["gap_extension_penalty"]  # -1

    aligner.mode = "global"
    for alignment in aligner.align(seq1, seq2):
        print("Score:", alignment.score)
        print(alignment)

In [ ]:
align_sequences_biopython(
    sample1_sequence1, sample1_sequence2, {"penalty_type": "linear", "gap_penalty": -1}
)

In [ ]:
align_sequences_biopython(
    sample1_sequence1,
    sample1_sequence2,
    {"penalty_type": "affine", "gap_penalty": -10, "gap_extension_penalty": -1},
)

### Your Output

Please run the cells below to present your final output.

#### Test Case 1

In [ ]:
seq1 = "PRTEINS"
seq2 = "PRTWPSEIN"

In [ ]:
alignment_score, alignment = align_sequences(
    seq1, seq2, "linear", {"scoring_matrix": scoring_matrix, "gap_penalty": -1}
)
print(alignment_score)
print(alignment)

In [ ]:
alignment_score, alignment = align_sequences(
    seq1,
    seq2,
    "affine",
    {"scoring_matrix": scoring_matrix, "gap_penalty": -10, "gap_extension_penalty": -1},
)
print(alignment_score)
print(alignment)